### Pre-requisite Steps

Preparation
0. Install python required libraries
1. Create data directory
2. Download credit card fraud detection dataset to local filesystem
3. Create google cloud storage bucket for project
4. Create google bigquery dataset
5. Upload credit card fraud detection dataset to bigquery dataset with auto detect schema load table
6. Create Dataflow to read data from bigquery and not to do any preprocessing and write into cloud storage bucket

#### Install required pre-requisite libraries

In [12]:
!pip install google-cloud-storage

In [13]:
!mkdir -p data

#### Import required libraries

In [14]:
import pandas as pd
from google.cloud import storage
from google.cloud import bigquery

In [40]:
# Set global variables
PROJECT = 'qwiklabs-gcp-33cc08e2c9cb0695'
BUCKET = 'qwiklabs-gcp-33cc08e2c9cb0695-projects'
DATASET = 'credit_card_fraud_detection'
TABLE = 'creditcard'
URI = 'https://storage.googleapis.com/advanced-solutions-lab/fraud/creditcard.csv'
GS_URI = 'gs://{}/credit-card-fraud-detection/creditcard-schema.csv'.format(BUCKET)

In [41]:
import os
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['DATASET'] = DATASET
os.environ['TABLE'] = TABLE
os.environ['URI'] = URI
os.environ['GS_URI'] = GS_URI

In [18]:
def create_storage_bucket(bucket_name):
    """Creates a new bucket."""
    storage_client = storage.Client()
    bucket = storage_client.create_bucket(bucket_name)
    print('Bucket {} created'.format(bucket.name))

In [19]:
create_storage_bucket(BUCKET)

Bucket qwiklabs-gcp-33cc08e2c9cb0695-projects created


#### Create Bigquery dataset

In [22]:
def create_bigquery_dataset(dataset_name):
  """Creates a new bigquery dataset."""
  # Get Client object
  bigquery_client = bigquery.Client()
  # create dataset reference object
  dataset_ref = bigquery_client.dataset(dataset_name)
  # Construct a full Dataset object to send to the API.
  dataset = bigquery.Dataset(dataset_ref)
  # Specify the geographic location where the dataset should reside.
  dataset.location = 'US'
  # Send the dataset to the API for creation.
  # Raises google.api_core.exceptions.AlreadyExists if the Dataset already
  # exists within the project.
  dataset = bigquery_client.create_dataset(dataset)
  print('Bigquery dataset {} created'.format(dataset_name))

In [25]:
create_bigquery_dataset(DATASET)

Bigquery dataset credit_card_fraud_detection created


#### Download credit card fraud detection dataset

In [28]:
!wget $URI -O ./data/creditcard-download.csv

--2018-11-12 16:01:07--  https://storage.googleapis.com/advanced-solutions-lab/fraud/creditcard.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.167.128, 2a00:1450:400c:c00::80
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.167.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 150828752 (144M) [text/csv]
Saving to: ‘./data/creditcard-download.csv’

./data/creditcard-d 100%[===================>] 143.84M   101MB/s    in 1.4s    

2018-11-12 16:01:09 (101 MB/s) - ‘./data/creditcard-download.csv’ saved [150828752/150828752]



In [34]:
!head $PWD/data/creditcard-download.csv

"Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount","Class"
0,-1.3598071336738,-0.0727811733098497,2.53634673796914,1.37815522427443,-0.338320769942518,0.462387777762292,0.239598554061257,0.0986979012610507,0.363786969611213,0.0907941719789316,-0.551599533260813,-0.617800855762348,-0.991389847235408,-0.311169353699879,1.46817697209427,-0.470400525259478,0.207971241929242,0.0257905801985591,0.403992960255733,0.251412098239705,-0.018306777944153,0.277837575558899,-0.110473910188767,0.0669280749146731,0.128539358273528,-0.189114843888824,0.133558376740387,-0.0210530534538215,149.62,"0"
0,1.19185711131486,0.26615071205963,0.16648011335321,0.448154078460911,0.0600176492822243,-0.0823608088155687,-0.0788029833323113,0.0851016549148104,-0.255425128109186,-0.166974414004614,1.61272666105479,1.06523531137287,0.48909501589608,-0.143772296441519,0.635558093258208,0.4639170410

In [29]:
df = pd.read_csv("./data/creditcard-download.csv", sep=",")

In [30]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [31]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.165980e-15,3.416908e-16,-1.373150e-15,2.086869e-15,9.604066e-16,1.490107e-15,-5.556467e-16,1.177556e-16,-2.406455e-15,...,1.656562e-16,-3.444850e-16,2.578648e-16,4.471968e-15,5.340915e-16,1.687098e-15,-3.666453e-16,-1.220404e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [32]:
df.to_csv('./data/creditcard-schema.csv', sep=',', encoding='utf-8', index=False)

In [33]:
!head $PWD/data/creditcard-schema.csv

Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0.0,-1.3598071336738,-0.0727811733098497,2.53634673796914,1.37815522427443,-0.33832076994251803,0.462387777762292,0.239598554061257,0.0986979012610507,0.363786969611213,0.0907941719789316,-0.551599533260813,-0.617800855762348,-0.991389847235408,-0.31116935369987897,1.46817697209427,-0.47040052525947795,0.20797124192924202,0.0257905801985591,0.403992960255733,0.251412098239705,-0.018306777944153,0.277837575558899,-0.110473910188767,0.0669280749146731,0.12853935827352803,-0.189114843888824,0.13355837674038698,-0.0210530534538215,149.62,0
0.0,1.1918571113148602,0.26615071205963,0.16648011335321,0.448154078460911,0.0600176492822243,-0.0823608088155687,-0.0788029833323113,0.0851016549148104,-0.255425128109186,-0.16697441400461402,1.6127266610547901,1.06523531137287,0.48909501589608,-0.143772296441519,0.635558093258208,0.463917041022171,-0.114804663102346,-0.18336127012399

In [35]:
%bash
gsutil -m rm -rf gs://${BUCKET}/credit-card-fraud-detection/*
gsutil -m cp data/*.csv gs://${BUCKET}/credit-card-fraud-detection/

CommandException: 1 files/objects could not be removed.
Copying file://data/creditcard-download.csv [Content-Type=text/csv]...
Copying file://data/creditcard-schema.csv [Content-Type=text/csv]...
\ [2/2 files][292.5 MiB/292.5 MiB] 100% Done                                    
Operation completed over 2 objects/292.5 MiB.                                    


In [38]:
def load_bigquery_table_from_uri(dataset_name, table_name, uri):
  """Loads data into bigquery table from uri source."""
  # Get Client object
  bigquery_client = bigquery.Client()
  
  dataset_ref = bigquery_client.dataset(dataset_name)
  job_config = bigquery.LoadJobConfig()
  job_config.autodetect = True
  job_config.skip_leading_rows = 1
  # The source format defaults to CSV, so the line below is optional.
  job_config.source_format = bigquery.SourceFormat.CSV

  load_job = bigquery_client.load_table_from_uri(
      uri,
      dataset_ref.table(table_name),
      job_config=job_config)  # API request
  print('Starting job {}'.format(load_job.job_id))

  load_job.result()  # Waits for table load to complete.
  print('Job finished.')

  destination_table = bigquery_client.get_table(dataset_ref.table(table_name))
  print('Loaded {} rows.'.format(destination_table.num_rows))

In [42]:
load_bigquery_table_from_uri(DATASET, TABLE, GS_URI)

Starting job 3fa4d5b2-573b-46c6-90c5-de4129c70cb3
Job finished.
Loaded 284807 rows.
